In [ ]:
import Pkg
Pkg.activate(".")
Pkg.status()

In [ ]:
import ImplicitAD as IAD
import LinearAlgebra as LA
import ForwardDiff
import ForwardDiff as FD
import ReverseDiff
import ReverseDiff as RD

In [ ]:
# import ChainRulesCore
import LineSearches
import Optim

In [ ]:
using Plots

In [ ]:
using BurgersEquation
import BurgersEquation as BE

In [ ]:
# using Revise

In [ ]:
# using ImplicitAD
# import ImplicitAD as IAD

# Parameters

In [ ]:
Tf = tf = 1.0
Nx = 2^8
cfl = 0.85
x0 = [0.25, 1.0]
;

In [ ]:
function make_gif(hist::BurgersEquation.BurgersHistory, N, gif_name; fps=20)

    # ani = @animate for ll in 1:length(optim_trace)
    #     make_plot_group(optim_trace, ll - 1, params, tol)
    # end

    x_grid = BurgersEquation.space_grid(N)

    umin = floor(minimum(hist.u[0]); digits=1)
    umax = ceil(maximum(hist.u[0]); digits=1)

    ani = @animate for k in sort(collect(keys(hist.t)))
        tu = hist.t[k]
        uk = hist.u[k]
        plot(x_grid, BE.expand_solution(uk), title="Time: $(tu)", legend=false, ylim=(umin,umax))
    end

    return gif(ani, gif_name * ".gif", fps=fps)

end

# Burgers Solver Tests

In [ ]:
function hat_initial_condition(x, hl, hr, hh, hb)
    hm = 0.5 * (hr + hl)
    if hl < x && x < hm
        v = (hh - hb) / (hm - hl) * (x - hl) + hb
    elseif hm <= x && x < hr
        v = (hb - hh) / (hr - hm) * (x - hr) + hb
    else
        v = hb*one(x)
    end
    return v
end

function set_initial_condition(ic, N)
    u0 = zeros(N)
    for i in 1:N
        xi = BurgersEquation.gridpoint(i, N)
        u0[i] = ic(xi)
    end
    return u0
end

In [ ]:
# f(u) = 0.5*u^2
# fu(u) = u
# Tf = tf = 1.0

# # ratio = 0.35 # ratio = dt / dx
# Nx = 2^10
# @show Nx
# dx = BurgersEquation.gridsize(Nx)
# # dt = 5e-3

# # u0 = collect(range(1.0, 0.0, Nx))
# # u0 = zeros(Nx)
# # mp = Int(round(Nx/2))
# # u0[mp-100:mp+100] .= 1.0
# # u0 = hat_initial_condition(0.25, 0.75, 0.5, 0.2, Nx)
# # u0 = set_initial_condition(x->hat_initial_condition(x, 0.25, 0.75, 0.5, 0.2), Nx)
# u0 = set_initial_condition(x->1+0.2*sin(2*pi*x), Nx)
# # u0 = set_initial_condition(x->0.75 - (0.5 - x)^2, Nx)

# umax = maximum(u0)
# # cfl = umax * dt / dx
# # @show cfl
# cfl = 0.85
# # dt = cfl * dx / umax
# # @show dt
# # dt = 1.0 / ceil(Tf / dt)
# # @show dt
# dt = 1.0 / ceil(Tf * umax / (cfl * dx))
# @show dt
# cfl = umax * dt / dx
# @show cfl

# save_rate = max(Int(floor(2e-3 / dt)), 1)
# @show save_rate
# ;

f(u) = 0.5*u^2
fu(u) = u

@show Nx
dx = BurgersEquation.gridsize(Nx)
u0 = set_initial_condition(x->1+0.2*sin(2*pi*x), Nx)
# u0 = collect(BurgersEquation.space_grid(Nx)[1:Nx] .- 0.5)

umax = maximum(u0)
dt = 1.0 / ceil(Tf * umax / (cfl * dx))
@show dt
Nt = Tf / dt
@show Nt
cfl = umax * dt / dx
@show cfl

save_rate = max(Int(floor(2e-3 / dt)), 1)
@show save_rate
;

In [ ]:
plf = BurgersEquation.setup(u0, f, fu, Tf, dt, Nx, :lf; save_rate=save_rate)
@time BurgersEquation.solve(plf)
plw = BurgersEquation.setup(u0, f, fu, Tf, dt, Nx, :lw; save_rate=save_rate)
@time BurgersEquation.solve(plw)

In [ ]:
plot(BurgersEquation.space_grid(Nx), BurgersEquation.expand_solution(plf.u0))
plot!(BurgersEquation.space_grid(Nx), BurgersEquation.expand_solution(plw.u0))

In [ ]:
plot(BurgersEquation.space_grid(Nx), BurgersEquation.expand_solution(plf.uk), label="Lax-Friedrichs")
plot!(BurgersEquation.space_grid(Nx), BurgersEquation.expand_solution(plw.uk), label="Lax-Wendroff")

In [ ]:
plot(BurgersEquation.space_grid(Nx), BurgersEquation.expand_solution(plf.uk - plw.uk))

In [ ]:
# make_gif(plf.hist, Nx, "Lax-Friedrichs"; fps=20)

In [ ]:
# make_gif(plw.hist, Nx, "Lax-Wendroff"; fps=20)

# Auto Differentiation

In [ ]:
function implicit_burger_step(u0, params)

    prob = params[:prob]
    u1 = similar(u0)
    f = prob.f
    fprime = prob.fprime
    nflux = prob.num_flux
    dt = prob.dt
    dx = prob.dx
    Nx = prob.Nx
    
    BurgersEquation.burger_step(u0, u1, f, fprime, nflux, dt, dx, Nx)

    prob.uk .= u1
    
    return u1

end

function residual_burger_step(r, u1, u0, params)

    prob = params[:prob]
    f = prob.f
    fprime = prob.fprime
    flux = prob.num_flux
    dt = prob.dt
    Nt = prob.Nt
    dx = prob.dx
    Nx = prob.Nx
    ratio = dt / dx

     # Periodic boundary conditions
    Fp = flux(f, fprime, u0[1], u0[2], ratio)
    Fm = flux(f, fprime, u0[Nx], u0[1], ratio)
    r[1] = u1[1] - u0[1] + ratio * (Fp - Fm)

    Fp = flux(f, fprime, u0[Nx], u0[1], ratio)
    Fm = flux(f, fprime, u0[Nx-1], u0[Nx], ratio)
    r[Nx] = u1[Nx] - u0[Nx] + ratio * (Fp - Fm)

    for i in 2:Nx-1
        Fp = flux(f, fprime, u0[i], u0[i+1], ratio)
        Fm = flux(f, fprime, u0[i-1], u0[i], ratio)
        r[i] = u1[i] - u0[i] + ratio * (Fp - Fm)
    end

    return

end

# wrap residual function in a explicit form for convenience and ensure type of r is appropriate
function residual_wrap(yw, xw, pw) 
    T = promote_type(eltype(xw), eltype(yw))
    # match type of input variables
    rw = zeros(T, length(yw))
    residual_burger_step(rw, yw, xw, pw)
    return rw
end

function residual_jacobian_y(r, u1, u0, p)
    return LA.I
end

In [ ]:
function set_dt(cfl, tf, u0, Nx)
    # Assumes u >= 0
    umax = maximum(u0)
    dx = BurgersEquation.gridsize(Nx)
    dt = 1.0 / ceil(Tf * umax / (cfl * dx))
    return dt
end

function my_burger_loop(
    prob::BurgersEquation.BurgersProblem, p;
    progress::Bool=false, mode::Symbol=:implicit
)

    prob.v .= prob.u0
    u0 = prob.v
    # u1 = prob.uk
    # f = prob.f
    # fprime = prob.fprime
    # nflux = prob.num_flux
    dt = prob.dt
    Nt = prob.Nt
    # dx = prob.dx
    # Nx = prob.Nx
    hist = prob.hist

    if progress
        pm = PM.Progress(Nt)
    end

    count = 0

    while count < Nt

        if mode == :direct
            u1 = implicit_burger_step(u0, p)
        elseif mode == :implicit
            u1 = IAD.implicit(implicit_burger_step, residual_burger_step, u0, p; drdy=residual_jacobian_y)
            prob.uk .= u1
        elseif mode == :svd
            u1 = IAD.implicit_svd(implicit_burger_step, residual_burger_step, u0, p; drdy=residual_jacobian_y)
            prob.uk .= u1
        else
            error("Unrecognized mode: $mode")
        end

        count += 1
        BurgersEquation.save_solution(hist, u1, count, count * dt)

        if progress
            PM.next!(pm)
        end

        # @show eltype(u0)
        # @show eltype(u1)

        u0 .= u1

    end

    if progress
        PM.finish!(pm)
    end

    return

end

function burger_solution(
    x, p; 
    save::Bool=false, progress::Bool=false, mode::Symbol=:normal
)
    
    f(u) = 0.5*u^2
    fu(u) = u

    u0 = initial_condition(x, p)
    # @show eltype(u0)
    Nx = p[:Nx]
    cfl = p[:cfl]
    Tf = p[:tf]
    # Assumes u >= 0
    umax = maximum(u0)
    dx = BurgersEquation.gridsize(Nx)
    # dt = cfl * dx / umax
    # dt = 1.0 / ceil(Tf / dt)
    dt = 1.0 / ceil(Tf * umax / (cfl * dx))

    save_rate = save ? max(1, Int(floor(2e-3 / dt))) : -1
    # cfl = dt * umax / dx

    if p[:mode] == :svd
        tol = get(p, :tol, 0.0)
        nsv = get(p, :nsv, 3)
        println("SVD Mode -- " * ( tol > 0.0 ? "tol: $tol" : "nsv: $nsv"))
    end

    bp = BurgersEquation.setup(u0, f, fu, p[:tf], dt, Nx, p[:flux]; save_rate=save_rate)
    p[:prob] = bp
    if mode == :normal
        BurgersEquation.solve(bp; progress=progress)
    else
        my_burger_loop(bp, p; progress=progress, mode=mode)
    end
    
    return bp

end

In [ ]:
function ic_smooth(x, a, b)
    return b + a*sin(2*pi*x)
end

function ic_hat(x, a, b)
    return hat_initial_condition(x, 0.0, 1.0, b, a)
end

function initial_condition(x, p)

    Nx = p[:Nx]
    ic = p[:ic]
    u0 = zeros(eltype(x), Nx)

    for i in 1:Nx
        xi = BurgersEquation.gridpoint(i, Nx)
        u0[i] = ic(xi, x...)
    end

    return u0

end

function cost_u(x, p)

    bp = burger_solution(x, p; save=false, mode=p[:mode])

    # umax = maximum(bp.uk)
    # umin = minimum(bp.uk)
    # return umin - umax

    return (bp.uk[end] - 1.0)^2

end

function cost_x(x, p)
    return 0.5*LA.norm(x, 2)^2
end

function cost(x, p)
    a = 1.0
    b = 1.0
    return a*cost_x(x,p) + b*cost_u(x, p)
end

## Functional Tests

In [ ]:
# x0 = [0.25, 1.0]

my_params = Dict(
    :mode => :normal,
    :Nx => Nx,
    :cfl => cfl,
    :tf => tf,
    :flux => :lf,
    :ic => ic_smooth,
    # :ic => ic_hat,
)

direct_params = copy(my_params)
direct_params[:mode] = :direct

implicit_params = copy(my_params)
implicit_params[:mode] = :implicit

k = log2(Nx)
m = 2^Int(ceil(k/2))
n = 2^Int(floor(k/2))
@show (m,n)

svd_params = copy(my_params)
svd_params[:forward_svd] = true
svd_params[:mode] = :svd
# svd_params[:nsv] = 8
svd_params[:tol] = 1e-1
svd_params[:matdim] = (m,n)

my_params

In [ ]:
cost(x0, my_params)

In [ ]:
cost(x0, direct_params)

In [ ]:
cost(x0, implicit_params)

In [ ]:
cost(x0, svd_params)

In [ ]:
@time FD.gradient(x->cost(x, my_params), x0)

In [ ]:
@time FD.gradient(x->cost(x, direct_params), x0)

In [ ]:
@time FD.gradient(x->cost(x, implicit_params), x0)

In [ ]:
# @time FD.gradient(x->cost(x, svd_params), x0)

In [ ]:
# @time RD.gradient(x->cost(x, my_params), x0)

In [ ]:
# @time RD.gradient(x->cost(x, direct_params), x0)

In [ ]:
@time RD.gradient(x->cost(x, implicit_params), x0)

In [ ]:
@time RD.gradient(x->cost(x, svd_params), x0)

In [ ]:
# 2-element Vector{Float64}:
#  0.2500492590133452
#  0.9964819604159605

In [ ]:
cx_svd = FD.gradient(x->cost(x, svd_params), x0)
cx_fd = FD.gradient(x->cost(x, my_params), x0)
@show LA.norm(cx_svd - cx_fd, Inf)
@show LA.norm(cx_svd - cx_fd, 2)

## Residual Test

In [ ]:
# # f(u) = 0.5*u^2
# # fu(u) = u

# Nx = 2^6
# @show Nx
# dx = BurgersEquation.gridsize(Nx)

# # u0 = set_initial_condition(x->hat_initial_condition(x, 0.25, 0.75, 0.5, 0.2), Nx)
# u0 = set_initial_condition(x->1+0.2*sin(2*pi*x), Nx)
# umax = maximum(u0)
# tf = cfl * dx / umax
# dt = tf
# @show dt

# # r = zeros(Nx)
# rtest = BurgersEquation.setup(u0, f, fu, tf, dt, Nx, :lf; save_rate=-1)
# @show rtest.Nt
# BurgersEquation.solve(rtest)
# r = residual_wrap(rtest.uk, rtest.u0, Dict(:prob => rtest))
# @show LA.norm(r, Inf)
# ;

# SVD Compression

In [ ]:
function vector_svd(u, m, n)
    @assert(length(u) == m*n)
    return LA.svd(reshape(u, m, n))
end

function vector_svd_sweep(u::AbstractVector)
    N = length(u)
    rd = Dict()
    for m in 1:Int(ceil(sqrt(N)))
    # for m in 1:N
        (n, r) = divrem(N, m)
        if r == 0
            rd[(m,n)] = vector_svd(u, m, n)
        end
    end
    return rd
end

function plot_svd_sweep_results(svds::Dict)
    p = plot(yscale=:log10, yticks=[10.0^k for k in -20:1:4], dpi=300, xlabel="Index", ylabel="Singular Value")
    # for ((m,n),svd) in pairs(svds)
    for (m,n) in sort(collect(keys(svds)))
        svd = svds[(m,n)]
        scatter!(p, svd.S, label="($m,$n)")
    end
    return p
end

## Hat IC

In [ ]:
my_params[:ic] = ic_hat
bp = burger_solution(x0, my_params; save=true)
plot(BurgersEquation.space_grid(Nx), BurgersEquation.expand_solution(bp.u0))
plot!(BurgersEquation.space_grid(Nx), BurgersEquation.expand_solution(bp.uk))

In [ ]:
u_sol = bp.uk
svds = vector_svd_sweep(u_sol)
plot_svd_sweep_results(svds)

## Smooth IC

In [ ]:
my_params[:ic] = ic_smooth
bp = burger_solution(x0, my_params; save=true)
plot(BurgersEquation.space_grid(Nx), BurgersEquation.expand_solution(bp.u0))
plot!(BurgersEquation.space_grid(Nx), BurgersEquation.expand_solution(bp.uk))

In [ ]:
u_sol = bp.uk
svds = vector_svd_sweep(u_sol)
plot_svd_sweep_results(svds)

In [ ]:
for (m,n) in sort(collect(keys(svds)))
    svd = svds[(m,n)]
    @show (m,n)
    @show svd.S
end

## Memory Usage

In [ ]:
function svd_dimensions(ngrid)
    k = log2(ngrid)
    m = 2^Int(ceil(k/2))
    n = 2^Int(floor(k/2))
    return (m,n)
end

function svd_mem(nsv, ngrid)
    # m = Int(sqrt(ngrid))
    # return nsv + 2*nsv*m
    (m,n) = svd_dimensions(ngrid)
    return nsv * (m + n + 1)
end

function svd_mem_sweep(ngrid)
    # m = Int(sqrt(ngrid))
    (m,n) = svd_dimensions(ngrid)
    nsvs = 1:min(m,n)
    mem_svd = zeros(Int, min(m,n))
    for nsv in nsvs
        mem_svd[nsv] = svd_mem(nsv, ngrid)
    end
    return (nsvs, mem_svd)
end

function svd_mem_savings()
    mem_savings = Dict{Int, Any}()
    for k in 2:14
        Nx = 2^k
        (nsvs, mem_svd) = svd_mem_sweep(Nx)
        saving = Nx * 8 / 1024 .- mem_svd * 8 / 1024
        mem_savings[Nx] = (nsvs, saving)
    end
    return mem_savings
end

function plot_savings(mem_save)
    p = plot(
        # xscale=:log10,
        ylabel="kB",
        xlabel="Number Singular Values",
        title="SVD compression vs standard (per step)",
    )
    for Nx in sort(collect(keys(mem_save)))
        (nsvs, save) = mem_save[Nx]
        plot!(p, nsvs, save, label=string(Nx))
    end
    return p
end

In [ ]:
(num_svs, mem_svd) = svd_mem_sweep(Nx)
p = plot(num_svs, mem_svd * 8 / 1024,
    legend=false,
    ylabel="kB",
    xlabel="Number Singular Values",
    title="SVD compression vs standard (per step)",
)
plot!(p, num_svs, fill(Nx * 8 / 1024, length(num_svs)))
plot!(p, num_svs, fill(Nx * 8 / 1024, length(num_svs)) - mem_svd * 8 / 1024, label="Savings")

In [ ]:
mr = svd_mem_savings()
plot_savings(mr)

## Effect on Solution

In [ ]:
x0 = [0.25, 1.0]
bp = burger_solution(x0, svd_params; save=true)
u0 = bp.u0
uf = bp.uk
usvd0 = IAD.SVDVector(u0, m, n, 1)
usvdf = IAD.SVDVector(uf, m, n, 1)
p = plot(BurgersEquation.space_grid(svd_params[:Nx]), BurgersEquation.expand_solution(u0), label="u0")
plot!(p, BurgersEquation.space_grid(svd_params[:Nx]), BurgersEquation.expand_solution(usvd0), label="usvd0")
display(p)
p = plot(BurgersEquation.space_grid(svd_params[:Nx]), BurgersEquation.expand_solution(uf), label="uf")
plot!(p, BurgersEquation.space_grid(svd_params[:Nx]), BurgersEquation.expand_solution(usvdf), label="usvdf")
display(p)
;

In [ ]:
# x0 = [0.25, 1.0]
# Ntest = 256
# tf = 1.0
# cfl = 0.85
# f(u) = 0.5*u^2
# fprime(u) = u
# xk = BurgersEquation.space_grid(Ntest)
# u0 = IAD.SVDVector([ic_smooth(x, x0...) for x in xk[1:end-1]], 1)[:]
# dt = set_dt(cfl, tf, u0, Ntest)
# bp = BurgersEquation.setup(u0, f, fprime, tf, dt, Ntest, :lf; save_rate=1)
# BurgersEquation.burger_loop(bp)
# # bp = BurgersEquation.setup()
# # bp = burger_solution(x0, svd_params; save=true)
# # u0 = bp.u0
# uf = bp.uk
# # usvd0 = IAD.SVDVector(u0, m, n, 1)
# # usvdf = IAD.SVDVector(uf, m, n, 1)
# p = plot(BurgersEquation.space_grid(Ntest), BurgersEquation.expand_solution(u0), label="u0")
# # plot!(p, BurgersEquation.space_grid(svd_params[:Nx]), BurgersEquation.expand_solution(usvd0), label="usvd0")
# display(p)
# p = plot(BurgersEquation.space_grid(Ntest), BurgersEquation.expand_solution(uf), label="uf")
# # plot!(p, BurgersEquation.space_grid(svd_params[:Nx]), BurgersEquation.expand_solution(usvdf), label="usvdf")
# display(p)
# ;

In [ ]:
# make_gif(bp.hist, Ntest, "temp"; fps=20)

## Residual Grid Dependence

In [ ]:
function svd_solution_residual_sweep(x0, params)

    bp = burger_solution(x0, params)
    uf = bp.uk
    
    ngrid = params[:Nx]
    # m = Int(sqrt(ngrid))
    (m,n) = svd_dimensions(ngrid)
    res = zeros(min(m,n))
    nsvs = 1:min(m,n)
    
    for nsv in nsvs
        usvdf = IAD.SVDVector(uf, m, n, nsv)
        res[nsv] = LA.norm(uf - usvdf)
    end
    
    return (nsvs, res)
end

function svd_solution_residual_grid_size_sweep(x0)
    the_params = Dict(
        :mode => :normal,
        :cfl => cfl,
        :tf => tf,
        :flux => :lf,
        :ic => ic_smooth,
    )
    results = Dict{Int,Any}()
    for k in 2:14
        Nx = 2^k
        the_params[:Nx] = Nx
        (nsvs, res) = svd_solution_residual_sweep(x0, the_params)
        results[Nx] = (nsvs, res)
    end
    return results
end

function plot_svd_residuals(results::Dict)
    p = plot(yscale=:log10, ylabel="L2 Residual", xlabel="Number of Singular Values")
    # for (Nx, (nsvs, res)) in pairs(results)
    for Nx in sort(collect(keys(results)))
        (nsvs, res) = results[Nx]
        scatter!(p, nsvs, res, label=string(Nx))
    end
    return p
end

In [ ]:
x0 = [0.25, 1.0]
# nsvs, residuals = svd_solution_residual_sweep(x0, my_params)
# plot(nsvs, residuals, yscale=:log10, ylabel="L2 Residual", xlabel="Number of Singular Values")
rd = svd_solution_residual_grid_size_sweep(x0)
plot_svd_residuals(rd)

# FFT Compression

In [ ]:
import FFTW

In [ ]:
fft_params = copy(my_params)
Nfft = 256
fft_params[:Nx] = Nfft
# fft_params[:ic] = ic_hat
x0 = [0.25, 1.0]
bp = burger_solution(x0, fft_params; save=true)
uf = bp.uk
uf_hat = FFTW.rfft(uf)
;

In [ ]:
s1 = scatter(uf)
s2 = scatter(abs.(uf_hat), yscale=:log10, label="Mag")
scatter!(s2, abs.(real.(uf_hat)) .+ 1e-16, label="Real Mag")
scatter!(s2, abs.(imag.(uf_hat)) .+ 1e-16, label="Imag Mag")
plot(s1, s2, size=(1000,400))

In [ ]:
tol = 1e-1
@show length(uf_hat)
ru = real.(uf_hat)
idx = abs.(ru) .< tol
@show sum(idx)
# @show length(ru)
ru[idx] .= 0.0
iu = imag.(uf_hat)
idx = abs.(iu) .< tol
@show sum(idx)
# @show length(iu)
iu[idx] .= 0.0
uf_twiddle = complex.(ru, iu)
uf_fft = FFTW.irfft(uf_twiddle, Nfft)
;

In [ ]:
p1 = plot(BurgersEquation.space_grid(fft_params[:Nx]), BurgersEquation.expand_solution(uf))
plot!(p1, BurgersEquation.space_grid(fft_params[:Nx]), BurgersEquation.expand_solution(uf_fft))
p2 = plot(BurgersEquation.space_grid(fft_params[:Nx]), BurgersEquation.expand_solution(uf - uf_fft))
plot(p1, p2, size=(1000,400))

In [ ]:
function fft_solution(x0, params, tol)

    bp = burger_solution(x0, params)
    uf = bp.uk
    Nfft = params[:Nx]
    res = zeros(length(tol))
    nterms = zeros(length(tol))

    uf_hat = FFTW.rfft(uf)
    ntot = Nfft / 2 + 1

    for (k,tol) in enumerate(tol)
        ru = real.(uf_hat)
        idx = abs.(ru) .< tol
        ru[idx] .= 0.0
        nc = length(ru) - sum(idx)
    
        iu = imag.(uf_hat)
        idx = abs.(iu) .< tol
        iu[idx] .= 0.0
        nc += length(iu) - sum(idx)

        nterms[k] = 0.5 * nc / ntot
    
        uf_twiddle = complex.(ru, iu)
        uf_fft = FFTW.irfft(uf_twiddle, Nfft)
        res[k] = LA.norm(uf - uf_fft, 2)
    end
    
    return (res, nterms)

end

function fft_solution_sweep(x0, tols)
    the_params = Dict(
        :mode => :normal,
        :cfl => cfl,
        :tf => tf,
        :flux => :lf,
        :ic => ic_smooth,
    )
    results = Dict{Int,Any}()
    for k in 2:14
        Nx = 2^k
        the_params[:Nx] = Nx
        # (nsvs, res) = svd_solution_residual_sweep(x0, the_params, tols)
        (res, nterms) = fft_solution(x0, the_params, tols)
        results[Nx] = (res, nterms)
    end
    return results
end

function plot_fft_sweep(results::Dict, tols)
    p = plot(yscale=:log10, ylabel="L2 Residual",
             xscale=:log10, xlabel="Tolerance",
             legend=:bottomright)
    q = plot(ylabel="Fraction of Terms Kept", #yscale=:log2, 
        xlabel="Tolerance", xscale=:log10)
    for Nx in sort(collect(keys(results)))
        (res, nterms) = results[Nx]
        scatter!(p, tols, res, label=string(Nx))
        scatter!(q, tols, nterms, label=string(Nx))
    end
    return plot(p, q, size=(1200,400))
end

In [ ]:
tol_sweep = [1e-1, 5e-2, 2e-2, 1e-2, 5e-3, 2e-3, 1e-3, 5e-4, 2e-4, 1e-4, 5e-5, 2e-5, 1e-5]
rd = fft_solution_sweep(x0, tol_sweep)
plot_fft_sweep(rd, tol_sweep)

# Wavelet Compression

In [ ]:
import Wavelets

In [ ]:
wav_params = copy(my_params)
Nwav = 1024
wav_params[:Nx] = Nwav
x0 = [0.25, 1.0]
bp = burger_solution(x0, wav_params; save=true)
uf = bp.uk
wt = Wavelets.wavelet(Wavelets.WT.db2)
uf_hat = Wavelets.dwt(uf, wt)
;

In [ ]:
s1 = plot(uf, legend=false)
s2 = scatter(abs.(uf_hat), yscale=:log10, legend=false)
plot(s1, s2, size=(1000,400))

In [ ]:
tol = 1e-2
# nwav = 128
wt = Wavelets.wavelet(Wavelets.WT.db2)
uf_hat = Wavelets.dwt(uf, wt)
idx = abs.(uf_hat) .< tol
@show length(uf_hat)
@show sum(idx)
uf_hat[idx] .= 0.0
# uf_hat[nwav+1:end] .= 0.0
uf_wav = Wavelets.idwt(uf_hat, wt)
;

In [ ]:
p1 = plot(BurgersEquation.space_grid(wav_params[:Nx]), BurgersEquation.expand_solution(uf))
plot!(p1, BurgersEquation.space_grid(wav_params[:Nx]), BurgersEquation.expand_solution(uf_wav))
p2 = plot(BurgersEquation.space_grid(wav_params[:Nx]), BurgersEquation.expand_solution(uf - uf_wav))
plot(p1, p2, size=(1000,400))

In [ ]:
function wav_solution(x0, params, tol)

    bp = burger_solution(x0, params)
    uf = bp.uk
    res = zeros(length(tol))
    nterms = zeros(length(tol))

    wt = Wavelets.wavelet(Wavelets.WT.db2)
    uf_what = Wavelets.dwt(uf, wt)
    nwav = params[:Nx]

    for (k,tol) in enumerate(tol)
        uf_hat = copy(uf_what)
        idx = abs.(uf_hat) .< tol
        uf_hat[idx] .= 0.0
        uf_wav = Wavelets.idwt(uf_hat, wt)
        res[k] = LA.norm(uf_wav - uf, 2)
        nterms[k] = (nwav - sum(idx)) / nwav
    end
    
    return (res, nterms)

end

function wav_solution_sweep(x0, tols)
    the_params = Dict(
        :mode => :normal,
        :cfl => cfl,
        :tf => tf,
        :flux => :lf,
        :ic => ic_smooth,
    )
    results = Dict{Int,Any}()
    for k in 2:14
        Nx = 2^k
        the_params[:Nx] = Nx
        # (nsvs, res) = svd_solution_residual_sweep(x0, the_params, tols)
        (res, nterms) = wav_solution(x0, the_params, tols)
        results[Nx] = (res, nterms)
    end
    return results
end

function plot_wav_sweep(results::Dict, tols)
    p = plot(yscale=:log10, 
        ylabel="L2 Residual", 
        # yticks=[10.0^k for k in -15:2:1],
        xscale=:log10, 
        xlabel="Tolerance",
        legend=:bottomright)
    q = plot(ylabel="Fraction of Terms Kept", xlabel="Tolerance",
        xscale=:log10)
    for Nx in sort(collect(keys(results)))
        (res, nterms) = results[Nx]
        idx = res .< 1e-16
        res[idx] .= 1e-16
        scatter!(p, tols, res, label=string(Nx))
        scatter!(q, tols, nterms, label=string(Nx))
    end
    return plot(p, q, size=(1200,400))
end

In [ ]:
tol_sweep = [1e-1, 5e-2, 2e-2, 1e-2, 5e-3, 2e-3, 1e-3, 5e-4, 2e-4, 1e-4, 5e-5, 2e-5, 1e-5]
# tol_sweep = [1e-1, 5e-2, 2e-2, 1e-2]
rd = wav_solution_sweep(x0, tol_sweep)
plot_wav_sweep(rd, tol_sweep)

# Time Comparison

In [ ]:
function svd_vector(uf, N)
    (m,n) = svd_dimensions(N)
    return LA.svd(reshape(uf, m, n))
end

In [ ]:
x0 = [0.25, 1.0]
tf = 1.0
cfl = 0.85
Ntime = 2^14 # ~16000

time_params = Dict(
    :mode => :normal,
    :Nx => Ntime,
    :cfl => cfl,
    :tf => tf,
    :flux => :lf,
    :ic => ic_smooth,
    # :ic => ic_hat,
)

bp = burger_solution(x0, time_params; progress=true)
uf = bp.uk
display(plot(BurgersEquation.space_grid(Nx), BurgersEquation.expand_solution(uf)))
@time svd_vector(uf, Ntime)
@time FFTW.rfft(uf)
@time Wavelets.dwt(uf, Wavelets.wavelet(Wavelets.WT.db2))
;

In [ ]:
svdt = 0.22 * 0.0065
Ntime * svdt / cfl

# Optimization

In [ ]:
tol = 1e-4
my_options = Optim.Options(
    g_abstol=tol,
    # g_reltol=tol,
    outer_g_abstol=tol,
    # outer_g_reltol=tol,
    store_trace=true,
    extended_trace=true,
    show_trace=true
)
# lb = [-1.0, -1.0, 0.0, 0.0]
# ub = [1.0, 1.0, Inf, Inf]
Nx = 2^8
cfl = 0.85
tf = 1.0

lb = [0.0, -Inf]
ub = [Inf, Inf]
my_params = Dict(
    :Nx => Nx,
    :cfl => cfl,
    :tf => tf,
    :flux => :lf,
    :ic => ic_smooth,
    :mode => :direct,
)
@show my_params
@show x0
rvs_params = copy(my_params)
rvs_params[:mode] = :implicit
svd_params = copy(my_params)
svd_params[:mode] = :svd
# svd_params[:nsv] = 8
svd_params[:tol] = 1e-5
svd_params[:forward_svd] = true
svd_params[:matdim] = (m,n)
;

## ForwardDiff

In [ ]:
my_objective(x) = cost(x, my_params)

@time res = Optim.optimize(
    my_objective,
    lb,
    ub,
    x0,
    # Optim.Fminbox(Optim.BFGS(linesearch=LineSearches.BackTracking(order=3))),
    Optim.Fminbox(Optim.BFGS()),
    my_options;
    autodiff = :forward, # uses ForwardDiff.jl
)
@show Optim.converged(res)
@show Optim.minimum(res)
;

In [ ]:
res

In [ ]:
@show Optim.converged(res)
@show Optim.minimum(res)
@show Optim.minimizer(res)
x_fwd = Optim.minimizer(res)
;

## ReverseDiff

In [ ]:
# rvs_objective(x) = cost(x, rvs_params)
# function rvs_gradient(g, x)
#     g .= RD.gradient(rvs_objective, x)
#     return
# end

# @time res = Optim.optimize(
#     rvs_objective,
#     rvs_gradient,
#     lb,
#     ub,
#     x0,
#     Optim.Fminbox(Optim.BFGS()),
#     my_options;
#     # autodiff = :forward, # uses ForwardDiff.jl
# )
# @show Optim.converged(res)
# @show Optim.minimum(res)
# ;

In [ ]:
# @show Optim.converged(res)
# @show Optim.minimum(res)
# @show Optim.minimizer(res)
# x_rvs = Optim.minimizer(res)
# ;

## Finite Differences

In [ ]:
# my_objective(x) = cost(x, my_params)

# @time res = Optim.optimize(
#     my_objective,
#     lb,
#     ub,
#     x0,
#     Optim.Fminbox(Optim.BFGS()),
#     my_options
# )
# @show Optim.converged(res)
# @show Optim.minimum(res)
# ;

In [ ]:
# @show Optim.converged(res)
# @show Optim.minimum(res)
# @show Optim.minimizer(res)
# x_fnt = Optim.minimizer(res)
# ;

## SVD Reverse

Currently using ForwardDiff in an equivalent way because it is faster

In [ ]:
svd_objective(x) = cost(x, svd_params)
function svd_gradient(g, x)
    RD.gradient!(g, svd_objective, x)
    return
end

@time res = Optim.optimize(
    svd_objective,
    # svd_gradient,
    lb,
    ub,
    x0,
    Optim.Fminbox(Optim.BFGS()),
    my_options;
    autodiff = :forward, # uses ForwardDiff.jl
)
@show Optim.converged(res)
@show Optim.minimum(res)
;

In [ ]:
res

In [ ]:
@show Optim.converged(res)
@show Optim.minimum(res)
@show Optim.minimizer(res)
x_svd = Optim.minimizer(res)
;

In [ ]:
svd_objective(x) = cost(x, svd_params)
function svd_gradient(g, x)
    # g .= RD.gradient(svd_objective, x)
    RD.gradient!(g, svd_objective, x)
    return
end

@time res = Optim.optimize(
    svd_objective,
    svd_gradient,
    lb,
    ub,
    x0,
    Optim.Fminbox(Optim.BFGS()),
    my_options;
    # autodiff = :forward, # uses ForwardDiff.jl
)
@show Optim.converged(res)
@show Optim.minimum(res)
;

In [ ]:
res

In [ ]:
res |> typeof |> fieldnames

In [ ]:
res.g_residual

In [ ]:
res.x_abschange

In [ ]:
res.stopped_by

## Optimal Solution Visual

In [ ]:
LA.norm(x_fwd - x_svd, Inf)

In [ ]:
LA.norm(x_fwd - x_svd, 2)

In [ ]:
# bp = burger_solution(x_fwd, my_params; save=true)
# make_gif(bp.hist, Nx, "optimal_solution"; fps=20)

In [ ]:
# bp = burger_solution(x_svd, svd_params; save=true)
# make_gif(bp.hist, Nx, "svd_optimal_solution"; fps=20)